<a href="https://colab.research.google.com/github/Jahan08/Ambertools-CP2K-MM-QM-Biomolecular-Simulation/blob/main/hERG-Meta-learning-Convolution-Neural-Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
!pip install kora
import kora.install.rdkit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.7 MB/s eta 0:00:00


In [51]:
! pip install rdkit --upgrade


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.cross_decomposition import PLSRegression
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

In [24]:
df = pd.read_csv("/content/hERG_IC50_nM_new.csv")
df

,assay_chembl_id,assay_description,assay_type,canonical_smiles,document_journal,molecule_chembl_id,Value,IC50_uM,Activity
0,CHEMBL841079,Inhibition of hERG currents Kv11.1,T,O=C1NCCN1CCN1CCC(c2cn(-c3ccc(F)cc3)c3ccc(Cl)cc...,J Med Chem,CHEMBL12713,14.0,0.140,No
1,CHEMBL691014,K+ channel blocking activity in human embryoni...,F,O=C(CCCN1CC=C(n2c(=O)[nH]c3ccccc32)CC1)c1ccc(F...,J Med Chem,CHEMBL1108,32.2,0.322,No
2,CHEMBL691014,K+ channel blocking activity in human embryoni...,F,COc1ccc(CCN(C)CCCC(C#N)(c2ccc(OC)c(OC)c2)C(C)C...,J Med Chem,CHEMBL6966,143.0,1.430,No
3,CHEMBL877203,K+ channel blocking activity in Chinese hamste...,F,CCCCN(CCCC)CCC(O)c1cc2c(Cl)cc(Cl)cc2c2cc(C(F)(...,J Med Chem,CHEMBL1107,196.0,1.960,No
4,CHEMBL691014,K+ channel blocking activity in human embryoni...,F,CCOC(=O)N1CCC(=C2c3ccc(Cl)cc3CCc3cccnc32)CC1,J Med Chem,CHEMBL998,173.0,1.730,No
...,...,...,...,...,...,...,...,...,...
2963,CHEMBL5048865,Inhibition of hERG by patch clamp method,T,CCOP(=O)(Cn1ccc(NC(=O)c2cc(Oc3ccc(S(C)(=O)=O)c...,J Med Chem,CHEMBL5081517,44000.0,440.000,Yes
2964,CHEMBL5048865,Inhibition of hERG by patch clamp method,T,CCOP(=O)(Cn1ccc(NC(=O)c2cc(Oc3ccc(S(=O)(=O)N4C...,J Med Chem,CHEMBL5072442,33000.0,330.000,Yes
2965,CHEMBL5049389,Inhibition of hERG,T,Cc1nc(C)c([C@H](OC(C)(C)C)C(=O)O)c(N2CCC(C)(C)...,J Med Chem,CHEMBL5093378,670.0,6.700,No
2966,CHEMBL5050750,Inhibition of human ERG,T,Cc1cnc(Nc2ccnn2C)nc1-c1cc2n(c1)C(=O)N([C@H](CO...,ACS Med Chem Lett,CHEMBL5070887,14.0,0.140,No


In [57]:
!pip install rdkit-pypi==2021.03.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement rdkit-pypi==2021.03.5 (from versions: 2021.9.2, 2021.9.2.1, 2021.9.3, 2021.9.4, 2021.9.5.1, 2022.3.1b1, 2022.3.1, 2022.3.1.1, 2022.3.2, 2022.3.2.1, 2022.3.3, 2022.3.4, 2022.3.5, 2022.9.1, 2022.9.2, 2022.9.3, 2022.9.4, 2022.9.5, 2023.3.1b1)
ERROR: No matching distribution found for rdkit-pypi==2021.03.5


In [58]:
!git clone https://github.com/rdkit/rdkit.git

Cloning into 'rdkit'...
remote: Enumerating objects: 83911, done.
remote: Counting objects: 100% (1106/1106), done.
remote: Compressing objects: 100% (845/845), done.
remote: Total 83911 (delta 344), reused 619 (delta 252), pack-reused 82805
Receiving objects: 100% (83911/83911), 195.11 MiB | 23.71 MiB/s, done.
Resolving deltas: 100% (64018/64018), done.
Updating files: 100% (4316/4316), done.


In [59]:
%cd rdkit

/content/rdkit


In [61]:
!python setup.py install

python3: can't open file '/content/rdkit/setup.py': [Errno 2] No such file or directory


In [62]:
!apt-get update

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:6 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:7 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [1,010 kB]
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,343 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:13 http://archive.ubuntu.com/ubuntu focal-upd

In [63]:
!apt-get install -y build-essential python3-dev python3-pip python3-venv libxrender1 libfontconfig1 libxtst6 libxi6

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libfontconfig1 is already the newest version (2.13.1-2ubuntu3).
libfontconfig1 set to manually installed.
libxi6 is already the newest version (2:1.7.10-0ubuntu1).
libxi6 set to manually installed.
libxrender1 is already the newest version (1:0.9.10-1).
libxrender1 set to manually installed.
python3-dev is already the newest version (3.8.2-0ubuntu2).
python3-dev set to manually installed.
build-essential is already the newest version (12.8ubuntu1.1).
The following additional packages will be installed:
  python-pip-whl python3-setuptools python3-wheel python3.8-venv
Suggested packages:
  python-setuptools-doc
The following NEW packages will be installed:
  libxtst6 python-pip-whl python3-pip python3-setuptools python3-venv
  python3-wheel python3.8-venv
0 upgraded, 7 newly installed, 0 to remove and 30 not upgraded.
Need to get 2,409 kB of archives.
After this operation, 5,017 kB of addition

In [64]:
!pip install rdkit-pypi==2021.03.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement rdkit-pypi==2021.03.5 (from versions: 2021.9.2, 2021.9.2.1, 2021.9.3, 2021.9.4, 2021.9.5.1, 2022.3.1b1, 2022.3.1, 2022.3.1.1, 2022.3.2, 2022.3.2.1, 2022.3.3, 2022.3.4, 2022.3.5, 2022.9.1, 2022.9.2, 2022.9.3, 2022.9.4, 2022.9.5, 2023.3.1b1)
ERROR: No matching distribution found for rdkit-pypi==2021.03.5


In [55]:
from rdkit.Chem import Pharm2D
from rdkit.Chem import Descriptors

In [65]:
# define the fingerprints and their respective parameters
fingerprints = {
    'PubChem': (AllChem.GetMorganFingerprintAsBitVect, [2, 1024]),
    'ECFP6': (AllChem.GetMorganFingerprintAsBitVect, [3, 2048]),
    'MACCS': (Chem.rdMolDescriptors.GetMACCSKeysFingerprint, []),
    'AP2D': (AllChem.GetHashedAtomPairFingerprintAsBitVect, [2048]),
    'Estate': (Chem.Pharm2D, []),
    'Hybrid': (lambda x: np.concatenate([AllChem.GetMorganFingerprintAsBitVect(x, 2, 1024),
                                          AllChem.GetHashedTopologicalTorsionFingerprintAsBitVect(x, 1024)]), []),
    'KR': (AllChem.GetHashedAtomPairFingerprintAsBitVect, [2048]),
    'Circle': (AllChem.GetMorganFingerprintAsBitVect, [3, 2048]),
    'FP4C': (lambda x: Chem.RDKFingerprint(x, fpSize=1024, minPath=1, maxPath=7, fpType="counts", useHs=True), []),
    'FP4': (lambda x: Chem.RDKFingerprint(x, fpSize=1024, minPath=1, maxPath=7, fpType="bitvector", useHs=True), []),
    'CKD': (Chem.rdMolDescriptors._CalcCrippenContribs, []),
    'CKDGraph': (Chem.rdmolops.GetAdjacencyMatrix, []),
}

In [19]:
# Prepare the input data for the machine learning models
y = df['Value'].values
X_fps = {}
for fp_name, (fp_func, fp_args) in fingerprints.items():
    X_fps[fp_name] = np.asarray([fp_func(Chem.MolFromSmiles(smi), *fp_args) for smi in df['canonical_smiles']])

In [13]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim